In [1]:
!pip install langchain
!pip install --upgrade h11 httpcore httpx
!pip install openai
!pip install sentence-transformers
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.3/205.3 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the

In [2]:
import os
#from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
)
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma



from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/iot.txt'  # Or the path to your Google Drive
if os.path.exists(file_path):
    print("File found!")
else:
    print("File not found. Please check the path.")


#load_dotenv()

# https://python.langchain.com/docs/modules/data_connection/vectorstores/

#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ['OPENAI_API_KEY'] = 'ENTER YOUR API KEY'




LANGUAGE_MODEL = "gpt-3.5-turbo-instruct"



template: str = """/
    You are a customer support specialist /
    question: {question}. You assist users with general inquiries based on {context} only/
    and  technical issues. /
    if you do not know the answer to a question, you can redirect the user to contact@support.com /
    """
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["question", "context"],
    template="{question}",
)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

model = ChatOpenAI()


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


def load_documents():
    """Load a file from path, split it into chunks, embed each chunk and load it into the vector store."""
    # load the document and split it into chunks
    loader = TextLoader("/content/iot.txt")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    return text_splitter.split_documents(documents)


def load_embeddings(documents, user_query):
    """Create a vector store from a set of documents."""
    # create the open-source embedding function
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    db = Chroma.from_documents(documents, embedding_function)

    # load it into Chroma
    return db.as_retriever()


def generate_response(retriever, query):
    """Generate a response from a retriever and a query."""
    pass
    # Create a prompt template using a template from the config module and input variables
    # representing the context and question.
    # create the prompt
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | chat_prompt_template
        | model
        | StrOutputParser()
    )
    return chain.invoke(query)


def query(query):
    documents = load_documents()
    print(f"Loaded {len(documents)} documents.")
    retriever = load_embeddings(documents, query)
    return generate_response(retriever, query)


response = query("What technologies have made IoT possible?")
print(response)

Mounted at /content/drive
File found!


Loaded 3 documents.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

The technologies that have made IoT (Internet of Things) possible include:

1. Low-cost, low-power sensor technology: Affordable and reliable sensors enable manufacturers to integrate them into everyday objects, making them IoT-enabled.

2. Connectivity: Various network protocols for the internet allow sensors to connect to the cloud and other devices, facilitating efficient data transfer.

3. Cloud computing platforms: The availability of cloud platforms allows businesses and individuals to access the necessary infrastructure for scaling up IoT applications without managing it themselves.

4. Machine learning and analytics: Advances in machine learning and analytics, combined with access to vast amounts of data stored in the cloud, enable businesses to gather insights quickly and easily from IoT-generated data.

5. Conversational artificial intelligence (AI): Neural networks and natural language processing (NLP) have made it possible to integrate conversational AI into IoT devices, ma